# UE: an illustration of fixed point formulation with MSA as the solution algorithm.

In [16]:
using DelimitedFiles:readdlm

## Problem setting.

<img src="data/Caliper_net.png" width="800">

In [17]:
netFile = "data/network.csv"
csvdata = readdlm(netFile, ',', header=true)
data = csvdata[1]
header = csvdata[2]

1×10 Matrix{AbstractString}:
 "tail"  "head"  "cap"  "length"  "fft"  …  "speedLimit"  "toll"  "link_type"

In [18]:
display(header)
display(data)

1×10 Matrix{AbstractString}:
 "tail"  "head"  "cap"  "length"  "fft"  …  "speedLimit"  "toll"  "link_type"

7×10 Matrix{Float64}:
 1.0  2.0  50.0  20.0  20.0  0.15  4.0  60.0  0.0  1.0
 1.0  4.0  50.0  20.0  20.0  0.15  4.0  60.0  0.0  1.0
 2.0  3.0  50.0  20.0  20.0  0.15  4.0  60.0  0.0  1.0
 2.0  5.0  50.0  20.0  20.0  0.15  4.0  60.0  0.0  1.0
 3.0  6.0  50.0  20.0  20.0  0.15  4.0  60.0  0.0  1.0
 4.0  5.0  50.0  20.0  10.0  0.15  4.0  60.0  4.0  1.0
 5.0  6.0  50.0  20.0  10.0  0.15  4.0  60.0  2.0  1.0

In [19]:
links = []
for l in zip(round.(Int, data[:,1]), round.(Int, data[:,2]))
    push!(links, l)
end
cap = data[:, 3]
fft = data[:, 5]
alpha = data[:, 6]
beta = data[:, 7]
dem = 100.0

100.0

In [20]:
# BPR function
function BPR(cap, fft, alpha, beta,  flow)
    return fft * (1 + alpha * (flow/cap) ^ beta)
end

BPR (generic function with 1 method)

In [21]:
# AoN choice probability
function AoN(cost)
    prob = zeros(size(cost))
    prob[argmin(cost)] = 1.0
    return prob
end

AoN (generic function with 2 methods)

In [22]:
# link-path incidence matrix (a 7 x 3 matrix)
display(links)
Δ = Matrix([
    1 1 0;
    0 0 1;
    1 0 0;
    0 1 0;
    1 0 0;
    0 0 1;
    0 1 1
    ])

7-element Vector{Any}:
 (1, 2)
 (1, 4)
 (2, 3)
 (2, 5)
 (3, 6)
 (4, 5)
 (5, 6)

7×3 Matrix{Int64}:
 1  1  0
 0  0  1
 1  0  0
 0  1  0
 1  0  0
 0  0  1
 0  1  1

## Formulation
It is a fixed point problem with respect to path flow. 
$$
\mathbf{f} = F(\mathbf{f}) \Rightarrow \mathbf{f = q * Pr(-\Delta^{T}t(\Delta f))}
$$

Alternatively, we can also write it as a fixed point problem with respect to link flow.
$$
\mathbf{x} = G(\mathbf{x}) \Rightarrow \mathbf{x = \Delta q * Pr(-\Delta^{T}t(x))}
$$

We use MSA method to solve this fixed point problem with respect to path flow. 

In [23]:
# intial value
f = [100; 0; 0]

3-element Vector{Int64}:
 100
   0
   0

In [24]:
# iteration 1
iter = 1
x = Δ * f
t = BPR.(cap, fft, alpha, beta, x)
c = Δ' * t
f_aux = AoN(c) * dem
f = (f_aux + iter * f) / (1 + iter)

3-element Vector{Float64}:
 50.0
  0.0
 50.0

In [25]:
# iteration 2
iter = 2
x = Δ * f
t = BPR.(cap, fft, alpha, beta, x)
c = Δ' * t
f_aux = AoN(c) * dem
f = (f_aux + iter * f) / (1 + iter)

3-element Vector{Float64}:
 33.333333333333336
  0.0
 66.66666666666667

In [26]:
# iteration 3
iter = 3
x = Δ * f
t = BPR.(cap, fft, alpha, beta, x)
c = Δ' * t
f_aux = AoN(c) * dem
f = (f_aux + iter * f) / (1 + iter)

3-element Vector{Float64}:
 25.0
 25.0
 50.0

In [27]:
# iteration 4
iter = 4
x = Δ * f
t = BPR.(cap, fft, alpha, beta, x)
c = Δ' * t
f_aux = AoN(c) * dem
f = (f_aux + iter * f) / (1 + iter)

3-element Vector{Float64}:
 20.0
 20.0
 60.0

In [28]:
# iteration 5
iter = 5
x = Δ * f
t = BPR.(cap, fft, alpha, beta, x)
c = Δ' * t
f_aux = AoN(c) * dem
f = (f_aux + iter * f) / (1 + iter)

3-element Vector{Float64}:
 16.666666666666668
 16.666666666666668
 66.66666666666667

In [29]:
# Let's do it for many loops.
f = [100; 0; 0]
f_aux = []
for iter in 1:10000
    f_aux = dem * AoN(Δ' * BPR.(cap, fft, alpha, beta, Δ * f))
    f = (f_aux  + iter * f) / (1 + iter)
end
c = Δ' * BPR.(cap, fft, alpha, beta, Δ * f)
println("f_aux = $f_aux")
println("f     = $f")
println("c     = $c")
println("x     = $(Δ*f)")

f_aux = [100.0, 0.0, 0.0]
f     = [19.488051194880615, 17.998200179982007, 62.51374862513749]
c     = [61.086294019887205, 61.0826546307144, 61.08045715131012]
x     = [37.48625137486262, 62.51374862513749, 19.488051194880615, 17.998200179982007, 19.488051194880615, 62.51374862513749, 80.51194880511949]


In [30]:
# Let's also do it link flow fixed point problem.
# Initial value
f = [0; 100; 0]
x = Δ*f
x_aux = []
for iter in 1:10000
    x_aux = Δ * dem * AoN(Δ' * BPR.(cap, fft, alpha, beta, x))
    x = (x_aux  + iter * x) / (1 + iter)
end
println("x_aux = $x_aux")
println("x     = $x")

x_aux = [100.0, 0.0, 100.0, 0.0, 100.0, 0.0, 0.0]
x     = [37.48625137486236, 62.51374862513749, 19.488051194880615, 17.998200179982007, 19.488051194880615, 62.51374862513749, 80.51194880511993]


## The auxiliary solution converges to the current solution.